In [4]:
# If you haven't already created an activated a virtual environment for this notebook, run this cell
!python3 -m venv venv
!source venv/bin/activate
!pip install -r requirements.txt

In [5]:
import scrapy
import os
current_dir = os.path.abspath('')
url = os.path.join(current_dir, "html/1992_World_Junior_Championships_in_Athletics_–_Men's_high_jump")
with open(url) as _f:
    url_data = _f.read()

response = scrapy.http.TextResponse(url, body=url_data, encoding='utf-8')

In [6]:
# Make sure that the interesting data is available 
table = response.xpath('//table')[1].xpath('tbody')
for tr in table.xpath('tr'):
    print(tr.xpath('td/b/text()').extract()[0],
          tr.xpath('td/a/text()').extract()[0]
    )

Gold Steve Smith
Silver Tim Forsyth
Bronze Takahiro Kimino


In [7]:
scrapped_data = {}
for tr in table.xpath('tr'):
    medal = tr.xpath('td/b/text()').extract()[0]
    athlete = tr.xpath('td/a/text()').extract()[0]
    scrapped_data[medal] = athlete

scrapped_data

{'Gold': 'Steve Smith', 'Silver': 'Tim Forsyth', 'Bronze': 'Takahiro Kimino'}

In [8]:
import json

# You can convert Python into JSON first, but there is no need if you use `json.dump()`
# as shown next
json_data = json.dumps(scrapped_data)

# Persist it in a file:
with open("1992_results.json", "w") as _f:
    # use dump() with the Python dictionary directly. 
    # the conversion is done on the fly
    json.dump(scrapped_data, _f)

In [9]:
# construct the data first

column_names = ["Medal", "Athlete"]
rows = []

for tr in table.xpath('tr'):
    medal = tr.xpath('td/b/text()').extract()[0]
    athlete = tr.xpath('td/a/text()').extract()[0]
    rows.append([medal, athlete])

In [10]:
# Now persist it to disk
import csv

with open("1992_results.csv", "w") as _f:
    writer = csv.writer(_f)

    # write the column names
    writer.writerow(column_names)

    # now write the rows
    writer.writerows(rows)

In [2]:
# Adding this data to a sqlite DB
import sqlite3

# Connect to DB (will create if non-existent)
conn = sqlite3.connect("1992_results.db")

# Create cursor object to interact with DB
curs = conn.cursor()

In [22]:
# Create a table
curs.execute(''' CREATE TABLE IF NOT EXISTS results 
             ( id INTEGER PRIMARY KEY, medal TEXT, athlete TEXT ) 
             ''')

# Commit the changes
conn.commit()

In [70]:
conn = sqlite3.connect("1992_results.db")
curs = conn.cursor()
query = 'INSERT INTO results(medal, athlete) VALUES(?, ?)'

# Loop for acquiring data to be added
for tr in table.xpath('tr'):
    medal = tr.xpath('td/b/text()').extract()[0]
    athlete = tr.xpath('td/a/text()').extract()[0]
    curs.execute(query, (medal, athlete))

# Commit changes
conn.commit()

In [3]:
# Lets have a look at the table
curs.execute('SELECT * FROM results')
rows = curs.fetchall()

# Print
for row in rows:
    print(row)

(1, 'Gold', 'Steve Smith')
(2, 'Silver', 'Tim Forsyth')
(3, 'Bronze', 'Takahiro Kimino')


In [68]:
# Deleting all records from the DB if required to start the practice again
curs.execute('''DELETE FROM results''')

conn.commit()

In [15]:
# Close connection (best practice after each query)
conn.close()